In [1]:
import numpy as np
import pandas as pd

In [18]:
train_df=pd.read_table("data/ratings_train.txt")
test_df=pd.read_table("data/ratings_test.txt")

In [5]:
train_df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [19]:
#Null 및 숫자를 공백 문자로 변경
#대부분의 문자는 re로 처리
#for루프를 가능하면 사용하지 않아야함.
import re

train_df["document"]=train_df["document"].fillna(" ") #밑에꺼 해주려면 결측값을 없애야함.
train_df['document'].apply(lambda x: re.sub(r"\d+", " ", x))#\d는 숫자를 의미, apply는 행방향으로 실행, for루프보다 빠름
test_df["document"]=test_df["document"].fillna(" ")
test_df["document"].apply(lambda x: re.sub(r"\d+", " ", x ))

0                                                      굳 ㅋ
1                                     GDNTOPCLASSINTHECLUB
2                    뭐야 이 평점들은.... 나쁘진 않지만  점 짜리는 더더욱 아니잖아
3                         지루하지는 않은데 완전 막장임... 돈주고 보기에는....
4         D만 아니었어도 별 다섯 개 줬을텐데.. 왜  D로 나와서 제 심기를 불편하게 하죠??
                               ...                        
49995            오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함
49996         의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO
49997                   그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다
49998       절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네
49999                                           마무리는 또 왜이래
Name: document, Length: 50000, dtype: object

In [10]:
from konlpy.tag import Okt #자바없이 할 수 있는 유일한 라이브러리, 버전이 0.0.6보다 높아야함.

In [20]:
okt=Okt() #객체만들기

In [12]:
def tw_tokenizer(text):
    tokenizer_ko = okt.morphs(text) #자소분류
    return tokenizer_ko

In [13]:
tw_tokenizer("가난한 내가 나타샤를 사랑해서")

['가난한', '내', '가', '나타', '샤를', '사랑', '해서']

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer=tw_tokenizer,
                        ngram_range=(1,2),
                        min_df=3, #3개이상이면 가져가고, 미만이면 버려라
                        max_df=0.9) #90%만 같으면 같은거라고 취급

In [22]:
tfidf.fit(train_df["document"])

c:\Users\user\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(max_df=0.9, min_df=3, ngram_range=(1, 2),
                tokenizer=<function tw_tokenizer at 0x000001CC186E8900>)

In [23]:
tfidf_matrix_train=tfidf.transform(train_df['document'])
tfidf_matrix_train


<150000x130333 sparse matrix of type '<class 'numpy.float64'>'
	with 3096525 stored elements in Compressed Sparse Row format>

In [24]:
from sklearn.linear_model import LogisticRegression
lg_clf = LogisticRegression(C=3.5, random_state=42)
lg_clf.fit(tfidf_matrix_train, train_df['label'])

LogisticRegression(C=3.5, random_state=42)

In [25]:
import joblib
joblib.dump(lg_clf, "movie_lr.pkl")
joblib.dump(tfidf, "movie_lr_tfidf.pkl")

['movie_lr_tfidf.pkl']

In [27]:
#역직렬화
movie_lr=joblib.load("movie_lr.pkl")
movie_lr_tfidf=joblib.load("movie_lr_tfidf.pkl")

In [35]:
q="힝~~ 노잼"
q=re.sub(r"\d+", " ", q)
q_matrix=movie_lr_tfidf.transform([q])

In [36]:
#모델 결과
movie_lr.predict(q_matrix)

array([0], dtype=int64)